In [10]:
import speech_recognition as sr
from pydub import AudioSegment

In [11]:
!pip install pydub

In [12]:
# Function to convert MP3 to WAV
def convert_mp3_to_wav(mp3_file, wav_file):
    audio = AudioSegment.from_mp3(mp3_file)
    audio.export(wav_file, format="wav")

In [13]:
# Function to transcribe audio and extract the medical information
def extract_medical_info(audio_file):
  # Convert MP3 to WAV (if necessary)
    if audio_file.endswith(".mp3"):
        wav_file = "converted_audio.wav"
        convert_mp3_to_wav(audio_file, wav_file)
        audio_file = wav_file

    # Initialize recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    # Recognize speech using Google Web Speech API
    try:
        text = recognizer.recognize_google(audio_data)
        print("Transcription: ", text)
    except sr.UnknownValueError:
        print("Could not understand the audio")
        return {}
    except sr.RequestError:
        print("Request failed, check your network connection")
        return {}

    # Define keywords for different symptoms
    symptoms = {
        "Pain Descriptions": ["burning", "stabbing", "pressure"],
        "Respiratory Issues": ["shortness of breath", "coughing"],
        "Neurological Issues": ["dizziness", "numbness", "headache"],
        "Infection Signs": ["fever", "chills", "swelling"]
    }

    # Create a dictionary to store matched symptoms
    medical_info = {key: [] for key in symptoms}

    # Check the transcribed text for each symptom keyword
    for category, keywords in symptoms.items():
        for keyword in keywords:
            if keyword in text.lower():
                medical_info[category].append(keyword)

    # Filter out categories with no matches
    medical_info = {category: values for category, values in medical_info.items() if values}

    return medical_info

In [14]:
# Usage example
audio_file = "dizz.mp3"  # Replace with your audio file path
medical_info = extract_medical_info(audio_file)
print("Extracted Information:", medical_info)

Transcription:  I have dizziness
Extracted Information: {'Neurological Issues': ['dizziness']}
